In [3]:
pip install transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 34.3 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
emotion_labels = ["anger","disgust","fear","joy","sadness","surprise","neutral"]


In [5]:
Cheat_sheet = {
    "anger": ["calm", "validate", "de_escalate"],
    "disgust": ["reassure", "support"],
    "fear": ["reassure", "protect"],
    "joy": ["celebrate", "share"],
    "sadness": ["comfort", "validate"],
    "surprise": ["explain", "react"],
    "neutral": ["acknowledge", "respond"]
}


In [6]:
import json

with open("/kaggle/input/notebookba7deda072/myprompts.json", "r") as f:
    myprompts = json.load(f)

In [7]:
import random

def myprompt_examples(emotion, n=6):
    if emotion not in myprompts:
        return ""

    examples = random.sample(
        myprompts[emotion],
        n
    )

    myprompt_text = ""
    for ex in examples:
        myprompt_text += f"""
User: {ex['User']}
Assistant: {ex['Assistant']}
"""

    return myprompt_text.strip()

In [8]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.pad_token = tokenizer.eos_token


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

2026-02-07 09:44:43.775450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770457483.973289      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770457484.031274      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770457484.484686      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770457484.484732      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770457484.484735      55 computation_placer.cc:177] computation placer alr

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [9]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)


In [10]:
from peft import LoraConfig, get_peft_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

for name, param in model.named_parameters():
    param.requires_grad = "lora" in name

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Trainable parameters:", trainable_params)

Trainable parameters: 13631488


In [18]:
from datasets import Dataset




flat_data = []
for req_emotion in emotion_labels:
    for item in myprompts.get(req_emotion, []):

        context = item.get("Context","").strip()

        if context != "":
            prompt_text = (
                f"Context: {context}\n"
                f"User: {item['User']}\n"
                f"Assistant:"
            )
        else:
            prompt_text = f"User: {item['User']}\nAssistant:"

        completion_text = f" {item['Assistant']}"

        flat_data.append({
            "prompt": prompt_text,
            "completion": completion_text,
            "emotion": req_emotion  
        })


dataset = Dataset.from_list(flat_data)

In [20]:
dataset = Dataset.from_list(flat_data)
print(f"Total samples: {len(dataset)}")

Total samples: 64591


In [21]:
dataset = dataset.shuffle(seed=42)

dataset = dataset.select(range(5000))

print("Using samples:", len(dataset))

Using samples: 5000


In [22]:
from collections import Counter

labels = [x["emotion"] for x in dataset]
counts = Counter(labels)

for k, v in counts.items():
    print(f"{k:10s} : {v}")

print("\nTotal:", sum(counts.values()))


joy        : 1140
neutral    : 995
disgust    : 566
fear       : 601
sadness    : 663
anger      : 780
surprise   : 255

Total: 5000


In [12]:

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

print("PAD token:", tokenizer.pad_token)
print("PAD token ID:", tokenizer.pad_token_id)

PAD token: </s>
PAD token ID: 2


In [13]:
def tokenize_function(examples):
    texts = [p + c for p, c in zip(examples["prompt"], examples["completion"])]
    tokenized = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=256,
    )
 
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized
tokenized_dataset = dataset.map(tokenize_function, batched=True)
print(tokenized_dataset[0])

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

{'prompt': "Context: Customer :Thank you! She drives an old cadillac and she drives it like a Ferrari. It's pretty grand watching her drive sometimes!\nAgent :\nUser: I was in awe of the driving ability of my old aunt, she's old but she still has it!\nAssistant:", 'completion': ' Haha... That sounds awesome. Wish I could carpool with her. I hate driving, but it sounds like she loves it.', 'input_ids': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 14268, 28747, 16648, 714, 15896, 368, 28808, 985, 19085, 396, 1571, 13150, 425, 323, 304, 630, 19085, 378, 737,

In [14]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq


In [15]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding=True,
    return_tensors="pt"
)


In [16]:
training_args = TrainingArguments(
    output_dir="./lora_mistral",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1.5,
    learning_rate=2e-4,
    warmup_ratio=0.05,
    fp16=True,
    logging_steps=50,
    save_steps=1000,
    save_total_limit=2,
    report_to="none",
    optim="paged_adamw_8bit",
    gradient_checkpointing=True
)



In [17]:
from transformers import Trainer

class CausalLMTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        if labels is None:
            raise ValueError("Labels are missing! Make sure your dataset produces 'labels'.")
        inputs["labels"] = labels.long()
        
        outputs = model(**inputs)
        loss = getattr(outputs, "loss", None)
        
        if loss is None:
            import torch
            logits = outputs.logits
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = inputs["labels"][..., 1:].contiguous()
            loss_fct = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
            loss = loss_fct(
                shift_logits.view(-1, shift_logits.size(-1)),
                shift_labels.view(-1)
            )
        return (loss, outputs) if return_outputs else loss


In [18]:
trainer = CausalLMTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)



/tmp/ipykernel_55/183362814.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CausalLMTrainer.__init__`. Use `processing_class` instead.
  trainer = CausalLMTrainer(


In [19]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
50,42.449700
100,40.593600
150,40.427900
200,40.406000
250,40.565600
300,40.301800
350,40.536100
400,40.276800
450,40.441900
500,40.488200


TrainOutput(global_step=938, training_loss=40.36018472439699, metrics={'train_runtime': 15418.3479, 'train_samples_per_second': 0.486, 'train_steps_per_second': 0.061, 'total_flos': 8.21156106951721e+16, 'train_loss': 40.36018472439699, 'epoch': 1.5008})

In [20]:
model.save_pretrained("/kaggle/working/emo_lora_adapter")
tokenizer.save_pretrained("/kaggle/working/emo_lora_adapter")

('/kaggle/working/emo_lora_adapter/tokenizer_config.json',
 '/kaggle/working/emo_lora_adapter/special_tokens_map.json',
 '/kaggle/working/emo_lora_adapter/chat_template.jinja',
 '/kaggle/working/emo_lora_adapter/tokenizer.model',
 '/kaggle/working/emo_lora_adapter/added_tokens.json',
 '/kaggle/working/emo_lora_adapter/tokenizer.json')

In [21]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_pro

In [22]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    torch_dtype=torch.float16,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, "./emo_lora_adapter")
tokenizer = AutoTokenizer.from_pretrained("./emo_lora_adapter")


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

NameError: name 'myprompts' is not defined